# CNMF Componenent evaluation

This is similar to the last visualization in `cnmf_viz.ipynb` but combines it with component evaluation

View cmaps with `r_values`, `cnn_preds`, `comp_SNR`

Click on components and press `"g"` to add to good, `"b"` to add to bad.

In [1]:
from mesmerize_core import *
import numpy as np
import pandas as pd

2023-07-28 11:10:32.858123: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from fastplotlib import ImageWidget, Plot, GridPlot
#from fastplotlib.utils import get_colors
#from fastplotlib.graphics.line_slider import LineSlider
from ipywidgets import VBox, IntSlider, Layout

In [4]:
pd.options.display.max_colwidth = 120

# Paths

In [5]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/Users/mirckuz/caiman_data/")

# batch path
batch_path = get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

## Load batch

In [6]:
df = load_batch(batch_path)
dfnew = df[df["algo"] == "cnmf"]

df = dfnew

In [7]:
# You can change this to plot the outputs for different batch items
index = 1

# get the motion corrected input movie as a memmap
cnmf_movie = df.iloc[index].caiman.get_input_movie()

# we can get the contours of the spatial components
contours, coms = df.iloc[index].cnmf.get_contours("all", swap_dim=False)

# and temporal components
temporal = df.iloc[index].cnmf.get_temporal("all")

ixs_good = df.iloc[index].cnmf.get_good_components()
ixs_bad = df.iloc[index].cnmf.get_bad_components()

cnmf_obj = df.iloc[index].cnmf.get_output()

In [8]:
np.sort(cnmf_obj.estimates.r_values)

array([-0.56382539, -0.50624548, -0.4904246 , ...,  0.95386888,
        0.95598139,  0.96759161])

In [9]:
rcm = df.iloc[index].cnmf.get_rcm()
corr_img = df.iloc[index].caiman.get_corr_image()

In [10]:
import fastplotlib
# 2 x 2 gridplot
# |=====================|
# |  movie |    rcm     |
# |---------------------|
# | corr  |  temporal   |
# |=====================|
cnmf_grid = GridPlot(
    shape=(2, 2), 
    controllers=[[0, 0], [0, 1]], 
    names=[["movie", "rcm"], ["corr", "temporal"]]
)

# movie and rcm, rcm is a lazy array and behaves similar to numpy arrays
movie_graphic = cnmf_grid["movie"].add_image(cnmf_movie[0], cmap="gnuplot2")
rcm_graphic = cnmf_grid["rcm"].add_image(rcm[0], cmap="gnuplot2")

# corr image
cnmf_grid["corr"].add_image(corr_img, cmap="viridis")

# contours for good components
contours_good, coms = df.iloc[index].cnmf.get_contours("good", swap_dim=False)

# random colors for contours and temporal components
# make an RGBA array for each color
rand_colors = np.random.rand(len(contours_good), 4)  # [n_contours, RGBA]
rand_colors[:, -1] = 1 # set alpha = 1

# get temporal of only good components
temporal_good = df.iloc[index].cnmf.get_temporal("good")

# add contours to both movie and rcm subplots
contours_movie = cnmf_grid["movie"].add_line_collection(contours_good, colors=rand_colors)
contours_rcm = cnmf_grid["rcm"].add_line_collection(contours_good, colors=rand_colors)

# line stack of temporal components
temporal_stack = cnmf_grid["temporal"].add_line_stack(temporal_good, colors=rand_colors, thickness=3.0, separate=15)

# plot single temporal, just like before
plot_temporal_single = Plot()
temporal_graphic = plot_temporal_single.add_line_collection(temporal_good, colors=rand_colors)

# since this is a GridPlot and not an ImageWidget we need to define sliders
slider = IntSlider(min=0, max=cnmf_movie.shape[0] - 1, value=0, step=1)

# vertical line sliders
#_ls = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max()), slider=slider)
#_ls2 = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max() + temporal_stack.items[-1].position.y), slider=slider)
#plot_temporal_single.add_graphic(_ls)
#cnmf_grid["temporal"].add_graphic(_ls2)

# function to update elach frame
def update_frame(change):
    ix = change["new"]
    movie_graphic.data = cnmf_movie[ix]
    rcm_graphic.data = rcm[ix]
    
slider.observe(update_frame, "value")

@plot_temporal_single.renderer.add_event_handler("resize")
def update_slider_width(*args):
    width, h = plot_temporal_single.renderer.logical_size
    slider.layout = Layout(width=f"{width}px")
    
VBox([plot_temporal_single.show(), cnmf_grid.show(), slider])

RFBOutputContext()

/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


RFBOutputContext()